# Sentiment Classification with Multinomial Naive Bayes

In [1]:
import os
import numpy as np
import math
from sklearn.model_selection import train_test_split

## Data and Features Extraction

In [2]:
data_path = 'datasets/data-tagged/'
classes = [0, 1]

In [3]:
def process_file(filepath):
    """Given a file, returns a list of tokens for that file"""
    x = []
    with open(filepath, 'r') as f:
        for l in f:
            # Filter lines which consist only of new line operator
            if l == '\n':
                continue
            
            token, pos_tagging = l.split('\t')
            x.append(token)
    return x

def preprocess_data(datapath, sentiment='POS'):
    idx = 0
    X = []
    y = []
    sentiment_value = 1 if sentiment == 'POS' else 0
    
    # For file in the folder
    current_path = datapath + sentiment
    for f in os.listdir(current_path):
        x = process_file(current_path + '/' + f)
        X.append(x)
        y.append(sentiment_value)

    return X, y

def get_dictionary(X):
    """Returns a dictionary which maps each token to its index in the feature space, along with a list of all features (tokens) in order"""
    idx = 0
    token_to_idx = {}
    features = []
    
    for x in X:
        for token in x:
            if token not in token_to_idx:
                token_to_idx[token] = idx
                idx += 1
                features.append(token)
    
    return token_to_idx, features

def featurize_data(X, features, token_to_idx):
    """Convert each sample from a list of tokens to a multinomial bag of words representation"""
    X_feat = []
    for x in X:
        x_feat = np.zeros((len(features)))
        for token in x:
            x_feat[token_to_idx[token]] += 1
        X_feat.append(x_feat)
    
    return X_feat

In [4]:
X_pos, y_pos = preprocess_data(data_path, 'POS')
X_neg, y_neg = preprocess_data(data_path, 'NEG')

In [5]:
# X = X_pos + X_neg
# y = y_pos + y_neg

In [6]:
# token_to_idx, features = get_dictionary(X)

In [7]:
# X = featurize_data(X, features, token_to_idx)

In [8]:
# assert(sum(X[0]) != 0)

## Multinomial Naive Bayes Model

In [92]:
class MultinomialNaiveBayes():
    def __init__(self, classes, num_feat, smoothing_value=0):
        # Number of features the model uses
        self.num_feat = num_feat
        # List of the classes
        self.classes = classes
        # Dictionary mapping each class to the prior probability p(C=c)
        self.class_to_prior = {c: 0 for c in classes}
        # self.class_to_feature_to_cond_prob[c][x] is used to store the estimate of the conditional probability p(X=x|C=c)
        self.class_to_feature_to_cond_prob = {c: np.zeros((num_feat,)) for c in classes}
        # A smoothing value of 0 is equivalent to no smoothing
        self.smoothing_value = smoothing_value
        
    def fit(self, X, y):
        y = np.array(y)
        X = np.array(X)
        # Computer priors
        for c in y:
            self.class_to_prior[c] += 1
        self.class_to_prior.update({c: self.class_to_prior[c] / len(y) for c in self.classes})
        
        # Compute estimate of the conditional probability p(X=x|C=c)
        for c in self.classes:
            X_c = X[y == c]
            features_frequencies = np.sum(X_c, axis=0)
            self.class_to_feature_to_cond_prob[c] = (features_frequencies + self.smoothing_value) / sum(features_frequencies + self.smoothing_value)
        
    def predict(self, X):
        return np.argmax(np.stack([self.compute_scores(X, c) for c in self.classes], axis=-1), axis=1)
    
    def compute_scores(self, X, c):
        return np.log(self.class_to_prior[c]) + np.matmul(X, np.log(self.class_to_feature_to_cond_prob[c]))
            
#     def compute_score(self, x, c):
#         # Compute score (unnormalized log probability) for given class
#         return np.log(self.class_to_prior[c]) + np.dot(x, np.log(self.smooth(self.class_to_feature_to_cond_prob[c])))
    
    def smooth(self, x, laplacian=False):
        x[x == 0.0] = 10 ** -15
        return x

## Test Model

In [93]:
# X = X_pos + X_neg
# y = y_pos + y_neg

In [94]:
# token_to_idx, features = get_dictionary(X)

In [95]:
# X = featurize_data(X, features, token_to_idx)

In [96]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [97]:
X = X_pos + X_neg

X_train = X_pos[:900] + X_neg[:900]
y_train = y_pos[:900] + y_neg[:900]

X_test = X_pos[900:] + X_neg[900:]
y_test = y_pos[900:] + y_neg[900:]

In [98]:
token_to_idx, features = get_dictionary(X)

X_train = featurize_data(X_train, features, token_to_idx)
X_test = featurize_data(X_test, features, token_to_idx)

In [101]:
%%time
model = MultinomialNaiveBayes(classes, len(features), smoothing_value=1)

model.fit(X_train, y_train)

CPU times: user 721 ms, sys: 730 ms, total: 1.45 s
Wall time: 1.63 s


In [102]:
%%time
y_pred = model.predict(X_test)
n_correct = sum(1 for i, _ in enumerate(y_pred) if y_pred[i] == y_test[i])

print("{0:.2f}% of sentences are correctly classified".format(n_correct * 100 / len(X_test)))

79.50% of sentences are correctly classified
CPU times: user 81.6 ms, sys: 37.2 ms, total: 119 ms
Wall time: 160 ms


## Comparison with Sklearn

In [61]:
%%time
from sklearn.naive_bayes import MultinomialNB

n_correct = 0

clf = MultinomialNB(alpha=1.0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
n_correct = sum(1 for i, _ in enumerate(y_pred) if y_pred[i] == y_test[i])

print("{0:.2f}% of sentences are correctly classified".format(n_correct * 100 / len(X_test)))

79.50% of sentences are correctly classified
CPU times: user 740 ms, sys: 410 ms, total: 1.15 s
Wall time: 971 ms
